In [1]:
import pandas as pd
import openai
from tqdm import tqdm
import os

In [2]:
#read csv and save in idf
idf = pd.read_csv('AiredaleEval.csv')
# Read openai api key from file apikey.txt
api_key_file = 'apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key

def evaluator_gpt(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=[{"role":"user", "content":prompt}],
        temperature=0.1
    )
    # Parse the response to get only the "content" part and save it in the response variable
    response = completion["choices"][0]["message"]
    return response

In [5]:
#add following columns to idf: RESPONSE_TOPIC, RESPONSE_SUBTOPIC, RESPONSE_CATEGORY, RESPONSE_GUIDANCE, RESPONSE_COMPLEXITY, RESPONSE_FEEDBACK
idf['RESPONSE_TOPIC'] = ''
idf['RESPONSE_SUBTOPIC'] = ''
idf['RESPONSE_CATEGORY'] = ''
idf['RESPONSE_GUIDANCE'] = ''
idf['RESPONSE_COMPLEXITY'] = ''
idf['RESPONSE_FEEDBACK'] = ''
#start a for loop to iterate through the rows of idf and add tqdm to show progress
for i in tqdm(range(len(idf))):
    #save the column 'MAIN_TOPIC' from current row in a variable called topic
    topic = idf['MAIN_TOPIC'][i]
    #save the column 'SUB_TOPIC' from current row in a variable called subtopic
    subtopic = idf['SUB_TOPIC'][i]
    #save the column 'CATEGORY' from current row in a variable called category
    category = idf['CATEGORY'][i]
    #save the column 'GUIDANCE' from current row in a variable called guidance
    guidance = idf['GUIDANCE'][i]
    #save the column 'PROMPT' from current row in a variable called text
    text = idf['PROMPT'][i]
    #write a prompt to check whether the text is relevant to the topic
    prompt_topic = "You are tasked to categorize a given text. You need to check whether the text provided below is relevant to the topic: " + topic + "." + "\n" + "Please answer 'y' for yes and 'n' for no. Do not say anything else" + "\n" + "Text: " + text + "\n"
    #write a prompt to check whether the text is relevant to the subtopic
    prompt_subtopic = "You are tasked to categorize a given text. You need to check whether the text provided below is relevant to the subtopic: " + subtopic + "." + "\n" + "Please answer 'y' for yes and 'n' for no. Do not say anything else" + "\n" + "Text: " + text + "\n"
    #write a prompt to check whether the text is relevant to the category
    prompt_category = "You are tasked to categorize a given text. You need to check whether the text provided below is relevant to the category: " + category + "." + "\n" + "Please answer 'y' for yes and 'n' for no. Do not say anything else" + "\n" + "Text: " + text + "\n"
    #write a prompt to check whether the text is relevant to the guidance
    prompt_guidance = "You are tasked to categorize a given text. You need to check whether the text provided below is relevant to the guidance: " + guidance + "." + "\n" + "Please answer 'y' for yes and 'n' for no. Do not say anything else" + "\n" + "Text: " + text + "\n"
    #write a prompt for complexity of text
    prompt_complexity = "You are tasked to evaluate the complexity of a given text. A text is complex if it is longer, elbaorate and well written with various instructions included. You need to check whether the text provided below is complex." + "\n" + "Please answer 'y' for yes and 'n' for no. Do not say anything else" + "\n" + "Text: " + text + "\n"
    #write a prompt for one sentence feedback
    prompt_feedback = "You are tasked to provide feedback for a given text. You need to provide feedback on how to make the question more elaborate and well written with various instructions included." + "\n" + "Please provide feedback in one sentence. Do not say anything else" + "\n" + "Text: " + text + "\n"
    #save the response from the evaluator_gpt function in a variable called response_topic
    response_topic = evaluator_gpt(prompt_topic)
    #save the response from the evaluator_gpt function in a variable called response_subtopic
    response_subtopic = evaluator_gpt(prompt_subtopic)
    #save the response from the evaluator_gpt function in a variable called response_category
    response_category = evaluator_gpt(prompt_category)
    #save the response from the evaluator_gpt function in a variable called response_guidance
    response_guidance = evaluator_gpt(prompt_guidance)
    #save the response from the evaluator_gpt function in a variable called response_complexity
    response_complexity = evaluator_gpt(prompt_complexity)
    #save the response from the evaluator_gpt function in a variable called response_feedback
    response_feedback = evaluator_gpt(prompt_feedback)
    #save the response_topic in the column 'RESPONSE_TOPIC' of the current row
    idf['RESPONSE_TOPIC'][i] = response_topic
    #save the response_subtopic in the column 'RESPONSE_SUBTOPIC' of the current row
    idf['RESPONSE_SUBTOPIC'][i] = response_subtopic
    #save the response_category in the column 'RESPONSE_CATEGORY' of the current row
    idf['RESPONSE_CATEGORY'][i] = response_category
    #save the response_guidance in the column 'RESPONSE_GUIDANCE' of the current row
    idf['RESPONSE_GUIDANCE'][i] = response_guidance
    #save the response_complexity in the column 'RESPONSE_COMPLEXITY' of the current row
    idf['RESPONSE_COMPLEXITY'][i] = response_complexity
    #save the response_feedback in the column 'RESPONSE_FEEDBACK' of the current row
    idf['RESPONSE_FEEDBACK'][i] = response_feedback
#save idf in a csv file called data_evaluated.csv
idf.to_csv('data_evaluated.csv', index=False)

  2%|▏         | 9/428 [01:21<1:10:15, 10.06s/it]